
# **Build a Chat UI with `gradio`**

본 튜토리얼에서는 빠르고 간단하게 ML 모델을 위한 웹 인터페이스를 제작 및 공유할 수 있는 Python 라이브러리인 gradio에 대해 소개합니다.

In [ ]:
!pip install -U gradio

### **1. Quick Start with Chat Interface**

gradio에서는 아주 간단하게 chatbot UI를 제작할 수 있도록, `ChatInterface` 클래스를 제공하고 있습니다.

먼저, 사용자가 입력한 메세지를 똑같이 따라하는 chatbot을 만드는 것부터 시작해볼까요?

In [ ]:
import gradio as gr

def echo(message, history):
    return "You typed: " + message

gr.ChatInterface(echo).launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://8f917fb0a1a55b7465.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)



> `gr.ChatInterface(echo).launch()`

위의 코드 한 줄로 깔끔한 chatbot UI가 만들어졌습니다. 좀 더 기능을 추가해볼까요?


만약 아주 긴 출력을 생성하는 chatbot을 만든다면, 모든 토큰을 처리한 후 한 번에 응답을 반환하는 것보다는 **원하는 출력 단위별로 응답을 보여주는 것(Streaming)**이 좋은 선택일 수 있습니다. 해당 기능을 구현하려면 다음과 같이 `yield`를 사용해 chat function을 작성하면 됩니다.


In [ ]:
import time

def stream_echo(message, history):
    for i in range(len(message)):
      time.sleep(0.01)
      yield "You typed: " + message[:i+1]

gr.ChatInterface(stream_echo).launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://48040626289097d6b0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


예시 문장:

Gradio is an open-source Python package that allows you to quickly build a demo or web application for your machine learning model, API, or any arbitary Python function. You can then share a link to your demo or web application in just a few seconds using Gradio's built-in sharing features. No JavaScript, CSS, or web hosting experience needed!
You'll notice that in order to make your first demo, you created an instance of the gr.Interface class. The Interface class is designed to create demos for machine learning models which accept one or more inputs, and return one or more outputs.



### **2. Integrate Friendli Serverless Endpoint**

이번에는 chat function을 Friendli Serverless Endpoint를 사용해 구현해보겠습니다. 사용할 모델은 [meta의 Llama-3-70B-Instruct](https://huggingface.co/meta-llama/Meta-Llama-3-70B-Instruct)입니다.

chat template:

```
{% set loop_messages = messages %}
  {% for message in loop_messages %}
    {% set content = '<|start_header_id|>' + message['role'] + '<|end_header_id|>\n\n'+ message['content'] | trim + '<|eot_id|>' %}
    {% if loop.index0 == 0 %}
      {% set content = bos_token + content %}
    {% endif %}
    {{ content }}
  {% endfor %}
  {% if add_generation_prompt %}
    {{ '<|start_header_id|>assistant<|end_header_id|>\n\n' }}
  {% endif %}
```

request_body:
```
{
  "messages" : [
    {
      "role" : "user",
      "content" : "Hello"
    },
    {
      "role" : "assistant",
      "content" : "Hello!, How can I help you?"
    },
    ...
  ]
}
```

튜토리얼 진행을 위해 Friendli Personal Access Token이 필요합니다.

다음 링크를 따라 미리 토큰을 발급한 후 아래의 `FRIENDLI_TOKEN`의 값을 채워주세요.
https://docs.friendli.ai/guides/serverless_endpoints/quickstart

Friendli Serverless Endpoint를 쉽게 사용하기 위해서,

Python package인 `friendli-client`를 설치하고, 해당 package에서 제공하는 chat API를 사용하겠습니다

In [ ]:
!pip install friendli-client

In [ ]:
from friendli import Friendli

client = Friendli(token=FRIENDLI_TOKEN)

def chat_function(message, history):
  new_messages = []
  for user, chatbot in history:
    new_messages.append({"role" : "user", "content": user})
    new_messages.append({"role" : "assistant", "content": chatbot})
  new_messages.append({"role": "user", "content": message})

  stream = client.chat.completions.create(
    model="meta-llama-3-70b-instruct",
    messages=new_messages,
    stream=True
  )
  res = ""
  for chunk in stream:
    res += chunk.choices[0].delta.content or ""
    yield res

gr.ChatInterface(chat_function).launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://095f6e10d2c070bcca.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


# **3. Share your app with gradio**

gradio를 사용하면 아주 간단한 방식으로 구현한 chatbot이나 ML 모델 app을 공유할 수 있습니다.

## 1. Generate Temporary Link

가장 간단한 방법으로, 코드의 한 줄만 변경해서 72시간 동안 공유할 수 있는 링크를 제공합니다.
- 해당 링크는 로컬에서 프로그램이 실행 중일 때만 유효합니다
- 해당 링크로 어떤 사람이든 접근할 수 있습니다. 보안 문제가 있다면 다음 섹션을 참고해주세요.

> Jupyter notebook을 사용하는 경우, defualt로 링크를 생성합니다. 그 외에는 `launch(share=True)`를 명시해주어야 합니다.

In [ ]:
import os
from friendli import Friendli

client = Friendli(token=FRIENDLI_TOKEN)

def chat_function(message, history):
  new_messages = []
  for user, chatbot in history:
    new_messages.append({"role" : "user", "content": user})
    new_messages.append({"role" : "assistant", "content": chatbot})
  new_messages.append({"role": "user", "content": message})

  stream = client.chat.completions.create(
    model="meta-llama-3-70b-instruct",
    messages=new_messages,
    stream=True
  )
  res = ""
  for chunk in stream:
    res += chunk.choices[0].delta.content or ""
    yield res

# gr.ChatInterface(chat_function).launch()
gr.ChatInterface(chat_function).launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://6ea486555c992baa6b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


## 2. Hosting on Huggingface Space

영구적으로 app을 호스팅하고 싶다면, [Huggingface Space](https://huggingface.co/spaces)을 사용할 수 있습니다.

튜토리얼 진행을 위해서
- space 생성을 위한 [Huggingface](https://huggingface.co/) 계정이 필요합니다.
- Jupyter Notebook 파일이 아닌 파이썬 파일(.py)로 작성된 프로그램이 필요합니다.

다음 파일을 다운받아 로컬 환경에서 진행해주세요
- https://drive.google.com/file/d/1XmV0Kh8RjKJ9KShf4wY50XJapBZNmxt6/view?usp=sharing
